# Notebook to Analyse Results from CNN Training

In [1]:
# Load needed packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
from sklearn.linear_model import LinearRegression

In [2]:
# Import result data and combine DF from both GDP metrics
res_dat = pd.read_csv("/Users/maxbehrens/Documents/Msc/Thesis/Data/results/cnn_results_final.csv")
print('Absolute DF: '+str(res_dat.shape))
res_dat_diff = pd.read_csv("/Users/maxbehrens/Documents/Msc/Thesis/Data/results/cnn_results_diff.csv")
print('Relative DF: '+str(res_dat_diff.shape))
res_dat = res_dat.append(res_dat_diff)
print('Combined DF: '+str(res_dat.shape))
res_dat.reset_index(drop=True, inplace=True)

Absolute DF: (75, 18)
Relative DF: (77, 18)
Combined DF: (152, 18)


# Filter by Validation MSE

## Results per Optimising Function

Just experimental as not very representative.

In [3]:
# Groupby Optimiser
res_dat.groupby(['prediction','optim'])['val_mse'].mean().reset_index().sort_values('val_mse')

,prediction,optim,val_mse
3,nuts_diff,sgd,0.402534
7,nuts_value,sgd,0.682682
6,nuts_value,rmsprob,5.105038
2,nuts_diff,rmsprob,8.221484
4,nuts_value,adam,12.708447
0,nuts_diff,adam,14.079164
5,nuts_value,adamax,38.195111
1,nuts_diff,adamax,192.523990


## Model Type

Transfer Models seem to work best.

In [4]:
# Groupby Model Type
res_dat.groupby(['prediction','transfer'])['val_mse'].mean().reset_index().sort_values(['val_mse'])

,prediction,transfer,val_mse
0,nuts_diff,inception_model,0.241636
2,nuts_diff,mobile_model,0.295845
5,nuts_value,inception_model,0.409371
1,nuts_diff,m_model,0.434973
7,nuts_value,mobile_model,0.514573
6,nuts_value,m_model,0.542444
8,nuts_value,s_model,6.384526
3,nuts_diff,s_model,10.276953
9,nuts_value,xs_model,62.654977
4,nuts_diff,xs_model,259.020890


## Learning Rate

In [5]:
# Groupby Learning Rate
res_dat.groupby(['prediction','lr'])['val_mse'].mean().reset_index().sort_values(['val_mse'])

,prediction,lr,val_mse
6,nuts_value,0.000100,1.753972
0,nuts_diff,0.000001,1.855402
4,nuts_value,0.000001,1.924946
5,nuts_value,0.000010,5.338823
1,nuts_diff,0.000010,8.728598
2,nuts_diff,0.000100,14.192922
7,nuts_value,0.001000,55.335412
3,nuts_diff,0.001000,201.255038


## Putting it all together

In [6]:
# Groupby all varied entities (Model, Optimiser, Learning Rate)
res_dat.groupby(['prediction','optim','transfer','lr'])['val_mse','val_mae'].mean().reset_index().sort_values(['val_mse'])

,prediction,optim,transfer,lr,val_mse,val_mae
10,nuts_diff,adam,mobile_model,0.00010,0.187100,0.314113
41,nuts_diff,rmsprob,inception_model,0.00010,0.196607,0.336306
48,nuts_diff,rmsprob,mobile_model,0.00001,0.197601,0.331402
6,nuts_diff,adam,m_model,0.00010,0.199481,0.339075
50,nuts_diff,rmsprob,mobile_model,0.00100,0.199883,0.329470
...,...,...,...,...,...,...
52,nuts_diff,rmsprob,s_model,0.00001,109.495255,3.313060
18,nuts_diff,adam,xs_model,0.00010,144.526199,6.704063
94,nuts_value,adam,xs_model,0.00100,221.912476,11.085407
114,nuts_value,adamax,xs_model,0.00100,696.742554,14.609852


## Get Minimum per Group to identify best config per model

In [7]:
# Groupby GDP Metric and Model to get best config per GDP and Model
res_dat.loc[res_dat.groupby(['prediction','transfer'])['val_mse'].idxmin()].sort_values('val_mse')

,date,no_layers,batch_size,epochs,res,dims,train_mse,train_mae,val_mse,val_mae,test_mse,test_mae,transfer,lr,optim,comment,type,prediction
141,2020-04-01,161,8,33,128,3,0.175943,0.258067,0.187100,0.314113,3.610731e+07,4426.158017,mobile_model,0.00010,adam,iterated nn,viirs_night,nuts_diff
128,2020-03-31,786,8,93,128,3,0.214842,0.141631,0.196607,0.336306,3.873773e+07,4529.964899,inception_model,0.00010,rmsprob,iterated nn,viirs_night,nuts_diff
109,2020-03-31,14,16,50,224,3,1.000012,0.402939,0.199481,0.339075,4.091630e+07,4837.077766,m_model,0.00010,adam,iterated nn,viirs_night,nuts_diff
77,2020-03-30,3,16,34,224,3,1.024195,0.402934,0.204136,0.335459,8.854131e+07,5736.178039,xs_model,0.00001,sgd,iterated nn,viirs_night,nuts_diff
99,2020-03-31,8,16,109,224,3,0.831297,0.382421,0.204280,0.333147,8.364756e+07,5341.106480,s_model,0.00010,rmsprob,iterated nn,viirs_night,nuts_diff
53,2020-03-31,786,8,34,224,3,0.167386,0.272442,0.340211,0.471769,1.796214e+08,11267.296640,inception_model,0.00001,rmsprob,iterated nn,viirs_night,nuts_value
31,2020-03-30,14,16,104,224,3,0.443608,0.497277,0.376948,0.506384,2.092537e+08,11950.476919,m_model,0.00001,sgd,iterated nn,viirs_night,nuts_value
2,2020-03-30,3,16,35,224,3,0.913748,0.596321,0.380223,0.503052,2.758528e+08,12802.620296,xs_model,0.00001,sgd,iterated nn,viirs_night,nuts_value
60,2020-03-31,161,8,49,224,3,0.210946,0.342324,0.393468,0.515830,1.734627e+08,11157.854450,mobile_model,0.00010,sgd,iterated nn,viirs_night,nuts_value
22,2020-03-30,8,16,42,224,3,1.000228,0.621141,0.402281,0.511368,1.634465e+08,10987.016705,s_model,0.00100,rmsprob,iterated nn,viirs_night,nuts_value


## Detailed Country Analysis

In [8]:
# Load prediction accuracy per country for relative GDP
relative_country = pd.read_csv("/Users/maxbehrens/Documents/Msc/Thesis/Data/results/result_country_relative.csv")
relative_country = relative_country.sort_values('mse')
relative_country.head()

,country,mae,mse,size
0,BG,605.282026,9.333309e+05,240
1,SK,655.497549,1.373092e+06,120
2,PL,1118.846502,1.634909e+06,120
5,CZ,1329.954475,2.265202e+06,240
4,ME,1310.228221,2.541235e+06,90


In [9]:
# Load prediction accuracy per country for absolute GDP
absolute_country = pd.read_csv("/Users/maxbehrens/Documents/Msc/Thesis/Data/results/result_country_abs.csv")
absolute_country = absolute_country.sort_values('mse')
absolute_country.head()

,country,mae,mse,size
8,HU,4535.263464,3.712200e+07,116
10,LT,5150.862809,4.156815e+07,116
6,ES,5425.947529,4.737486e+07,464
9,IT,6733.844866,6.713411e+07,348
7,FR,6666.207202,6.872155e+07,435


In [10]:
# Load country codes
country_codes = pd.read_csv("/Users/maxbehrens/Documents/Msc/Thesis/Data/gdp_data/country_codes.csv", delimiter=';')
country_codes.head()

,Code,Country
0,BE,Belgium
1,BG,Bulgaria
2,CZ,Czechia
3,DK,Denmark
4,DE,Germany


In [11]:
# Load Absolute GDP predictions
abs_preds = pd.read_csv("/Users/maxbehrens/Documents/Msc/Thesis/Data/results/result_preds_abs.csv")

## More analysis

In [12]:
# Standard Dev. per Country
country_std = abs_preds.groupby('country')['test_true_vals'].std().reset_index().sort_values('test_true_vals')
country_std.columns = ['country','std']
country_std.head()

,country,std
12,ME,331.896126
14,PL,397.792824
17,SK,422.389682
1,BG,468.551004
11,LV,781.191954


In [13]:
# Average per Country
country_mean = abs_preds.groupby('country')['test_true_vals'].mean().reset_index()
country_mean.columns = ['country','mean']
country_mean.head()

,country,mean
0,AT,39800.0
1,BG,5487.5
2,CZ,14025.0
3,DE,39602.5
4,DK,38437.5


In [14]:
# Merge prediction results with general country stats
relative_country_stats = relative_country.merge(country_std, on='country',how='left').merge(country_mean, on='country',how='left')
absolute_country_stats = absolute_country.merge(country_std, on='country',how='left').merge(country_mean, on='country',how='left')

In [15]:
# Check output
relative_country_stats.head(20)

,country,mae,mse,size,std,mean
0,BG,605.282026,9.333309e+05,240,468.551004,5487.500000
1,SK,655.497549,1.373092e+06,120,422.389682,13775.000000
2,PL,1118.846502,1.634909e+06,120,397.792824,9175.000000
3,CZ,1329.954475,2.265202e+06,240,956.986160,14025.000000
4,ME,1310.228221,2.541235e+06,90,331.896126,5966.666667
5,PT,1263.235610,2.545178e+06,480,1553.778141,16718.750000
6,RO,1464.334218,3.174598e+06,240,1537.867965,6562.500000
7,LV,1386.681310,3.347000e+06,120,781.191954,12700.000000
8,TR,1735.905234,3.954965e+06,480,791.595806,6781.250000
9,NL,2516.569192,1.044407e+07,360,3100.187082,40025.000000


In [16]:
# Save results to csv
relative_country_stats.to_csv("/Users/maxbehrens/Documents/Msc/Thesis/Data/results/result_relative_country_std.csv", index=False)
absolute_country_stats.to_csv("/Users/maxbehrens/Documents/Msc/Thesis/Data/results/result_absolute_country_std.csv", index=False)

## Convert results into latex table output

In [17]:
# For Absolute GDP Predictions
for index, row in absolute_country_stats.sort_values('mse').iterrows():
    print(country_codes.loc[country_codes['Code']==row['country'],'Country'].values[0]+" & \multicolumn{1}{c}{"+'{:,}'.format(round(row["mse"]))+"} & \multicolumn{1}{c}{"+'{:,}'.format(round(row['mae']))+"} & \multicolumn{1}{c}{"+'{:,}'.format(round(row['size']/29))+"} & \multicolumn{1}{c}{"+'{:,}'.format(round(row['mean']))+"} & \multicolumn{1}{c}{"+'{:,}'.format(round(row['std']))+"} \\\ [1.1ex]")

Hungary & \multicolumn{1}{c}{37,121,999} & \multicolumn{1}{c}{4,535} & \multicolumn{1}{c}{4} & \multicolumn{1}{c}{23,700} & \multicolumn{1}{c}{1,478} \\ [1.1ex]
Lithuania & \multicolumn{1}{c}{41,568,150} & \multicolumn{1}{c}{5,151} & \multicolumn{1}{c}{4} & \multicolumn{1}{c}{19,425} & \multicolumn{1}{c}{1,212} \\ [1.1ex]
Spain & \multicolumn{1}{c}{47,374,860} & \multicolumn{1}{c}{5,426} & \multicolumn{1}{c}{16} & \multicolumn{1}{c}{20,525} & \multicolumn{1}{c}{3,333} \\ [1.1ex]
Italy & \multicolumn{1}{c}{67,134,113} & \multicolumn{1}{c}{6,734} & \multicolumn{1}{c}{12} & \multicolumn{1}{c}{22,233} & \multicolumn{1}{c}{6,120} \\ [1.1ex]
France & \multicolumn{1}{c}{68,721,548} & \multicolumn{1}{c}{6,666} & \multicolumn{1}{c}{15} & \multicolumn{1}{c}{27,313} & \multicolumn{1}{c}{3,360} \\ [1.1ex]
Portugal & \multicolumn{1}{c}{69,297,673} & \multicolumn{1}{c}{6,836} & \multicolumn{1}{c}{16} & \multicolumn{1}{c}{16,719} & \multicolumn{1}{c}{1,554} \\ [1.1ex]
United Kingdom & \multicolumn{1}

In [19]:
# For Relative GDP Predictions
for index, row in relative_country_stats.sort_values('mse').iterrows():
    print(country_codes.loc[country_codes['Code']==row['country'],'Country'].values[0]+" & \multicolumn{1}{c}{"+'{:,}'.format(round(row["mse"]))+"} & \multicolumn{1}{c}{"+'{:,}'.format(round(row['mae']))+"} & \multicolumn{1}{c}{"+'{:,}'.format(round(row['size']/30))+"} & \multicolumn{1}{c}{"+'{:,}'.format(round(row['mean']))+"} & \multicolumn{1}{c}{"+'{:,}'.format(round(row['std']))+"} \\\ [1.1ex]")

Bulgaria & \multicolumn{1}{c}{933,331} & \multicolumn{1}{c}{605} & \multicolumn{1}{c}{8} & \multicolumn{1}{c}{5,488} & \multicolumn{1}{c}{469} \\ [1.1ex]
Slovakia & \multicolumn{1}{c}{1,373,092} & \multicolumn{1}{c}{655} & \multicolumn{1}{c}{4} & \multicolumn{1}{c}{13,775} & \multicolumn{1}{c}{422} \\ [1.1ex]
Poland & \multicolumn{1}{c}{1,634,909} & \multicolumn{1}{c}{1,119} & \multicolumn{1}{c}{4} & \multicolumn{1}{c}{9,175} & \multicolumn{1}{c}{398} \\ [1.1ex]
Czechia & \multicolumn{1}{c}{2,265,202} & \multicolumn{1}{c}{1,330} & \multicolumn{1}{c}{8} & \multicolumn{1}{c}{14,025} & \multicolumn{1}{c}{957} \\ [1.1ex]
Montenegro & \multicolumn{1}{c}{2,541,235} & \multicolumn{1}{c}{1,310} & \multicolumn{1}{c}{3} & \multicolumn{1}{c}{5,967} & \multicolumn{1}{c}{332} \\ [1.1ex]
Portugal & \multicolumn{1}{c}{2,545,178} & \multicolumn{1}{c}{1,263} & \multicolumn{1}{c}{16} & \multicolumn{1}{c}{16,719} & \multicolumn{1}{c}{1,554} \\ [1.1ex]
Romania & \multicolumn{1}{c}{3,174,598} & \multicolum